In [ ]:
import sys
sys.path.insert(0, '/Users/timkrausz/Documents/GitHub/DA_maze_paper/DA/Modules/')

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
plt.style.use("seaborn-poster")
from multi_rat_da import *
from hexLevelAnalyses import *
from photometryQuantifications import *
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
%matplotlib qt

In [2]:
hexData = PhotRats(None)
# the directory containing your dataframe
loadpath = "/Volumes/Tim/Photometry/10MfRatDataSet/data4sharing/"
datName = "hexLevelDf"
hexData.df = reduce_mem_usage(pd.read_csv(loadpath+datName+".csv",index_col=[0]))
hexData.directory_prefix = loadpath

Memory usage of dataframe is 54.99 MB
Memory usage after optimization is: 12.60 MB
Decreased by 77.1%


In [3]:
hexData.df = hexData.df.loc[(hexData.df.rat!="IM-1292"),]
seshsToRemove = [90, 92, 94]
hexData.df = hexData.df.loc[~(hexData.df.session.isin(seshsToRemove)),]
hexData.df.reset_index(inplace=True)

## run all processing steps to organize data and compute relevant variables

In [10]:
create_tridf(hexData)

get_portHist(hexData)

get_back2samePortTrials(hexData)

add_rewardLagColumns2Df(hexData,lags=5)

add_pathSpecificRwdLagColumns2Df(hexData,lags=5)

add_pathSimilarity2df(hexData)

hexData.triframe = hexData.tridf

add_prevRwdTakenAndAlt(hexData)

add_altPathSpecificRwdLagColumns2Df(hexData,lags=1)

### create dataframe to test difference between same-path and alternative-path DA

In [12]:
ratSameValtPreCpDA = create_sameValtPreCpDARatMeans(hexData)

sameVals = [ratSameValtPreCpDA[rat]['DARwdVsOmSame'] for rat in ratSameValtPreCpDA.keys()]
altVals = [ratSameValtPreCpDA[rat]['DARwdVsOmAlt'] for rat in ratSameValtPreCpDA.keys()]

### check if same-path DA is significantly different from alternative-path DA

In [ ]:
wilcoxon(sameVals,altVals)

### visualize results and print p-values

In [21]:
fig = plot_DAdifSameValt(hexData,ratSameValtPreCpDA)

44.0 0.00390625
44.0 0.00390625


### save regression results for analysis in R

In [24]:
hexData.df.to_csv(hexData.directory_prefix+"hexDf_with_PreCp_and_prevRwdAltSame_correctAlignment.csv")